In [16]:
import random
import numpy as np
import pandas as pd
from joblib import dump, load
np.random.seed(42)
random.seed(42)

In [8]:
data_dir = './data/'
with np.load('data/Xy.npz') as f:
    X = f['X']
    y = f['y']

In [9]:
# Perform temporal split of data into train/test sets
pop = pd.read_csv(data_dir + 'population/d10_with_vitals.csv').set_index('BMT_ID')

split_date = 201701001
split_idx = -85

assert (pop[:split_idx].index < split_date).all()
assert (pop[split_idx:].index >= split_date).all()

In [14]:
from sklearn import preprocessing, model_selection, metrics, utils
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
from joblib import Parallel, delayed
from sklearn.base import clone

In [11]:
# Specify hyperparameters and cv parameters
base_estimator = LogisticRegression(penalty='l2', class_weight='balanced', solver='liblinear')
param_grid = {
    'C': [10. ** n for n in range(-6, 7)],
    'penalty': ['l2'],
}

## Train model with baseline+vitals

In [12]:
Xtr, Xte = X[:split_idx], X[split_idx:]
ytr, yte = y[:split_idx], y[split_idx:]

cv_splits, cv_repeat = 5, 20
cv = model_selection.RepeatedStratifiedKFold(cv_splits, cv_repeat, random_state=0)
clf = model_selection.GridSearchCV(
    clone(base_estimator), param_grid, 
    cv=cv, scoring='roc_auc', n_jobs=5,
)
clf.fit(Xtr, ytr)
test_score = metrics.roc_auc_score(yte, clf.decision_function(Xte))

In [15]:
y_true = yte
y_score = clf.decision_function(Xte)

def boostrap_func(i, y_true, y_score):
    yte_true_b, yte_pred_b = utils.resample(y_true, y_score, replace=True, random_state=i)
    return metrics.roc_curve(yte_true_b, yte_pred_b), metrics.roc_auc_score(yte_true_b, yte_pred_b)

roc_curves, auc_scores = zip(*Parallel(n_jobs=4)(delayed(boostrap_func)(i, y_true, y_score) for i in tqdm(range(1000), leave=False)))
print('Test AUC: {:.3f} ({:.3f}, {:.3f})'.format(np.median(auc_scores), np.percentile(auc_scores, 2.5), np.percentile(auc_scores, 97.5)))

Test AUC: 0.658 (0.536, 0.784)
Test AUC: 0.659 ± 0.063


In [17]:
dump(clf, 'data/model_combined.joblib') 

['data/model_combined.joblib']

## Train model with baseline features only

In [19]:
Xtr, Xte = X[:split_idx, :52], X[split_idx:, :52]
ytr, yte = y[:split_idx], y[split_idx:]

cv_splits, cv_repeat = 5, 20
cv = model_selection.RepeatedStratifiedKFold(cv_splits, cv_repeat, random_state=0)
clf = model_selection.GridSearchCV(
    clone(base_estimator), param_grid, 
    cv=cv, scoring='roc_auc', n_jobs=5,
)
clf.fit(Xtr, ytr)
test_score = metrics.roc_auc_score(yte, clf.decision_function(Xte))

/data4/tangsp/venv/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [20]:
y_true = yte
y_score = clf.decision_function(Xte)

def boostrap_func(i, y_true, y_score):
    yte_true_b, yte_pred_b = utils.resample(y_true, y_score, replace=True, random_state=i)
    return metrics.roc_curve(yte_true_b, yte_pred_b), metrics.roc_auc_score(yte_true_b, yte_pred_b)

roc_curves, auc_scores = zip(*Parallel(n_jobs=4)(delayed(boostrap_func)(i, y_true, y_score) for i in tqdm(range(1000), leave=False)))
print('Test AUC: {:.3f} ({:.3f}, {:.3f})'.format(np.median(auc_scores), np.percentile(auc_scores, 2.5), np.percentile(auc_scores, 97.5)))

Test AUC: 0.512 (0.364, 0.643)


In [21]:
dump(clf, 'data/model_baseline.joblib') 

['data/model_baseline.joblib']

## Train model with vitals features only

In [22]:
Xtr, Xte = X[:split_idx, 52:], X[split_idx:, 52:]
ytr, yte = y[:split_idx], y[split_idx:]

cv_splits, cv_repeat = 5, 20
cv = model_selection.RepeatedStratifiedKFold(cv_splits, cv_repeat, random_state=0)
clf = model_selection.GridSearchCV(
    clone(base_estimator), param_grid, 
    cv=cv, scoring='roc_auc', n_jobs=5,
)
clf.fit(Xtr, ytr)
test_score = metrics.roc_auc_score(yte, clf.decision_function(Xte))

In [23]:
y_true = yte
y_score = clf.decision_function(Xte)

def boostrap_func(i, y_true, y_score):
    yte_true_b, yte_pred_b = utils.resample(y_true, y_score, replace=True, random_state=i)
    return metrics.roc_curve(yte_true_b, yte_pred_b), metrics.roc_auc_score(yte_true_b, yte_pred_b)

roc_curves, auc_scores = zip(*Parallel(n_jobs=4)(delayed(boostrap_func)(i, y_true, y_score) for i in tqdm(range(1000), leave=False)))
print('Test AUC: {:.3f} ({:.3f}, {:.3f})'.format(np.median(auc_scores), np.percentile(auc_scores, 2.5), np.percentile(auc_scores, 97.5)))

Test AUC: 0.633 (0.507, 0.757)


In [24]:
dump(clf, 'data/model_vitals.joblib') 

['data/model_vitals.joblib']